In [1]:
import numpy as np
import cupy as cp
from numba import cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_normal_float32
from time import perf_counter
size = (1024, 2, 1000)

In [2]:
# With numpy
%timeit np.random.normal(size=size)
# 59.1 ms ± 1.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

63.2 ms ± 296 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
# With cupy
%timeit cp.random.normal(size=size, dtype=cp.float32); cp.cuda.Device().synchronize()
# 48.4 µs ± 112 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)

136 µs ± 13.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
# With numba.cuda
@cuda.jit
def numba_cuda_normal(rng_states, out):
    pos = cuda.grid(1)
    for i in range(out.shape[2]):
        out[pos, 0, i] = xoroshiro128p_normal_float32(rng_states, pos)
        out[pos, 1, i] = xoroshiro128p_normal_float32(rng_states, pos)

threads_per_block = 8 # Best performance reach for threads_per_block=8 with RTX3080
blocks = size[0]//threads_per_block
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=1)
out = np.zeros(size, dtype=np.float32)
out_gpu = cuda.to_device(out)
numba_cuda_normal[blocks, threads_per_block](rng_states, out_gpu) # warmup
%timeit numba_cuda_normal[blocks, threads_per_block](rng_states, out_gpu); cuda.synchronize()
# 701 µs ± 869 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)

745 µs ± 2.75 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
# Modified version for numba.cuda

# This kernel maps thread index x to shape[0] and y to shape[2], so that more
# threads can be launched in parallel. The loop over shape[2] is replaced with
# the second thread index
@cuda.jit
def numba_cuda_normal_2(rng_states, out):
    pos, i = cuda.grid(2)
    
    # Ensure our thread is within the bounds of the array
    if pos < out.shape[0] and i < out.shape[2]:
        out[pos, 0, i] = xoroshiro128p_normal_float32(rng_states, pos * out.shape[2] + i)
        out[pos, 1, i] = xoroshiro128p_normal_float32(rng_states, pos * out.shape[2] + i)

# 256 threads / 8 warps per block - a reasonable, slightly arbitrary choice
threads_per_block = (16, 16) 

# Launch enough blocks for all data points.
# Sometimes this will launch slightly more blocks than needed -
# the calculation could be improved slightly
blocks = ((size[0] // threads_per_block[0]) + 1, (size[2] // threads_per_block[1]) + 1)

# RNG state initialization
rng_states = create_xoroshiro128p_states(size[0] * size[2], seed=1)

# Create output array on GPU and warm up JIT
out = np.zeros(size, dtype=np.float32)
out_gpu = cuda.to_device(out)
numba_cuda_normal_2[blocks, threads_per_block](rng_states, out_gpu) # warmup

# How many iterationss to loop through when timing?
N_ITERATIONS = 10000

# Timing: we launch all kernels then synchronize after launching all kernels
# before recording the end timer - this way we avoid including one sync
# per iteration in our timing.

start = perf_counter()

for i in range(N_ITERATIONS):
    numba_cuda_normal_2[blocks, threads_per_block](rng_states, out_gpu)

cuda.synchronize()
end = perf_counter()


# Iteration time is total time dividide by number of iterations
# Time is given in seconds, so multiply to get microseconds
iteration_time = ((end - start) / N_ITERATIONS) * 1_000_000

print(f"Time is {iteration_time} µs per iteration")

Time is 157.67731770001774 µs per iteration
